In [1]:
import obspy
from time import gmtime, strftime

def lowercaseInput(dct):# convert source input information to lowercase
    dct['datasource']= str(dct['datasource']).lower()
    dct['eventsource']= str(dct['eventsource']).lower()
    dct['magSource']= str(dct['magSource']).lower()
    dct['locationSource']= str(dct['locationSource']).lower()   
    
def updateDictionary(dct):# add calculated values to dictionary
    dct['dataid'] = dct['eventsource'] + dct['eventid']
    dct['eventPid'] = 'quakeml:' + dct['datasource'] + '.anss.org/event/'+ dct['dataid']
    dct['epPid'] = 'quakeml:' + dct['datasource'] + '.anss.org/eventParameters/'+ dct['eventid']
    dct['magPid'] = 'quakeml:' + dct['datasource'] + '.anss.org/magnitude/'+ dct['dataid'] + '/' + dct['type']
    dct['oPid'] = 'quakeml:' + dct['datasource'] + '.anss.org/origin/'+ dct['dataid']
    
def generateCreationInfo(dct): # create, fill and return obspy creation_info object 
    creation_info = obspy.core.event.base.CreationInfo(
        agency_id = dct['eventsource'],
        creation_time = strftime("%Y-%m-%dT%H:%M:%SZ", gmtime())
    )
    return creation_info
    
def depthToMeters(dct):
    depth = float(dct['depth'])
    depth = depth * 1000
    dct ['depth'] = str(depth)
    horizontal_uncertainty = float(dct['horizontalUncertainty'])
    horizontal_uncertainty = horizontal_uncertainty * 1000
    dct['horizontalUncertainty'] = str(horizontal_uncertainty)
    uncertainty = float(dct['uncertainty'])
    uncertainty = uncertainty * 1000
    dct ['uncertainty'] = str(uncertainty)

def generateOrigin(dct, ci):
    """ Create an origin object, fill it with items,
    set public id and add creation info. Then return
    origin object
    """
    new_origin = obspy.core.event.origin.Origin(
        resource_id = dct['oPid'],
        time = dct['time'],
        longitude = dct['longitude'], 
        latitude = dct['latitude'], 
        depth = dct['depth'],
        depth_errors = dct['uncertainty'],
        depth_type = dct['depthType'],
        evaluation_mode = dct['evaluationModeOrg'], 
        evaluation_status = dct['evaluationStatusOrg'],
        creation_info = ci,
        origin_uncertainty = dct['horizontalUncertainty'])
    return new_origin
    
def generateMagnitude(dct, ci, origin):
    """ Create a magnitude object, fill it with items,
    set public id and origin id pointing to origin
    and add creation info. Then return magnitde object
    """
    new_magnitude = obspy.core.event.magnitude.Magnitude(
        resource_id = obspy.core.event.base.ResourceIdentifier(
            id= dct['magPid']
        ),
        mag = dct['mag'],
        magnitude_type = dct['type'],
        evaluation_mode = dct['evaluationModeMag'], 
        evaluation_status = dct['evaluationStatusMag'],
        origin_id = dct['oPid'],
        # following block is equivalent to origin_id = dct['oPid'],
        #    origin_id = obspy.core.event.base.ResourceIdentifier(
        #       referred_object = origin
        #    ),
        creation_info = ci
    )
    return new_magnitude

def generateEvent(dct, ci, orgs, mags):# create fill and return obspy event object
    """ Create an event object and fill it with items
    including magnitudes, origins, creation info and
    event descriptions
    """
    eventDescriptions = []
    if dct['title'] != "": # check for title, if title not null add to quakeml
        event_title = obspy.core.event.event.EventDescription(text = dct['title'], type = "earthquake name" ) 
        eventDescriptions.append(event_title)
    new_event = obspy.core.event.event.Event(
        resource_id = dct['eventPid'],
        origins = orgs,
        magnitudes = mags,
        creation_info = ci,
        comments = [
            obspy.core.event.base.Comment(
                text = dct['comment'],
                force_resource_id = False
            )
        ],
        event_descriptions = eventDescriptions,
        event_type = dct['eventType'],
        preferred_magnitude_id = dct['magPid'],
        preferred_origin_id = dct['oPid']
    )
    return new_event

In [17]:
import argparse

def txt2dictionary (file,output):
    # create parser and add arguments
    #parser = argparse.ArgumentParser()
    #parser.add_argument('file', type=argparse.FileType('r'), help = "must input a .txt file.  see input.txt for example.  Further documentation is in README.md")
    #parser.add_argument('-o', '--output',  help = "output file, if not entered will default to 'out' + eventid + '.xml'")
    #args = parser.parse_args() # create arguments list
    
    # read in file to python dictionary
    infile = open(file, "r")
    lines = infile.readlines()
    data = {}
    for line in lines:
        if line[0] == "#" or line[0] == "" or line[0] == "\n":
            pass
        else:
            temp = line.split(":::")
            tmp = temp[1]
            temp[1] = tmp[:-1]
            data[temp[0]] = temp[1]
        output_filename = ""

    # Check for output file argument.
    # If no output file argument, make an output filename.
  #  if args.output:
    output_filename = output
  #  else:           #
   #     print ("No output file argument. Default filename will be generated...")
   #     output_filename = "out_" + data['datasource'].lower() + data['eventid'] + '.xml' # set output filename
    return data , output_filename

In [20]:
import utils as h
#import text_as_input as p
import obspy

data, output_filename = txt2dictionary('input.txt','outfile') # import file to python dictionary and get output filename
lowercaseInput(data) # make case-sensitive input lowercase
updateDictionary(data) # generate calculated fields
depthToMeters(data)

# Generate quakeml with obspy
magnitudes, origins = [], [] # lists for magnitudes and origins
creation_info = generateCreationInfo(data) # create creation_info object
origin = generateOrigin(data, creation_info) # create origin object
origins.append(origin)
magnitude = generateMagnitude(data, creation_info, origin) # create magnitude object
magnitudes.append(magnitude)
event = generateEvent(data, creation_info, origins, magnitudes) # create event object

# Add ANSS specific attributes
extra = { 'eventsource': {'value': data['eventsource'],
                          'type': 'attribute',
                          'namespace': 'http://anss.org/xmlns/catalog/0.1'},
          'dataid': {'value': data['dataid'],
                          'type': 'attribute',
                          'namespace': 'http://anss.org/xmlns/catalog/0.1'},
          'eventid': {'value': data['eventid'],
                          'type': 'attribute',
                          'namespace': 'http://anss.org/xmlns/catalog/0.1'},                              
          'datasource': {'value': data['datasource'],
                          'type': 'attribute',
                          'namespace': 'http://anss.org/xmlns/catalog/0.1'}}
event.extra = extra # add attributes to event
catalog = obspy.core.event.Catalog(events=[event], resource_id = data['epPid']) # create catalog object

# Write event if new or updated using obspy write
catalog.write(output_filename, format='QUAKEML',
              nsmap={'catalog': 'http://anss.org/xmlns/catalog/0.1'})

print("file was exported to: " + output_filename)

file was exported to: outfile
